Proyecto 2 - probabilidad y estadistica 

- Miguel Angel Nava Perez
- Ana Luisa Parra Valencia

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats 
from statsmodels.sandbox.stats.runs import runstest_1samp 
from statsmodels.formula.api import ols
import statsmodels.api as sm 
from tabulate import tabulate
from statsmodels.stats.multicomp import pairwise_tukeyhsd


def test_normalityKS(data, variable): # Pruaba de Normalidad Kolmogorov-Smirnof 
    """
    data: arreglo de datos a evaluar la normalidad
    variable: string con el nombre de la variable 
    """  
    print(f"\n Análisis de normalidad por Kolmogorov-Smirnov para '{variable}'")

    # Kolmogorov-Smirnov (KS) test
    ks_stat, ks_p = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
    print(f" Estadístico = {ks_stat:.4f}, p-valor = {ks_p:.4f}")

def test_normalitySW(data, variable): # Prueba de Normalizas Shapiro-Wilks 
    """
    data: arreglo de datos a evaluar la normalidad
    variable: string con el nombre de la variable 
    """
    print(f"\n Análisis de normalidad por Shapiro-Wilk para '{variable}'")
    # Shapiro-Wilk test
    shapiro_stat, shapiro_p = stats.shapiro(data)
    print(f"Estadístico = {shapiro_stat:.4f}, p-valor = {shapiro_p:.4f}")
    
def random_test(residuos):
    """
    Parameters
    ----------
    residuos : Array
        DESCRIPTION: Residuos del ANOVA 

    Returns
    -------
    None.

    """
    _, p_runs = runstest_1samp(residuos, correction=True)

    print(f"Prueba de Runs: p-valor={p_runs}")
    
def test_homogeneityL(var1, var2, name1, name2): # Prueba de levene
    """
    var1 y var2: variables a las que se corroborará homocedasticidad 
    name1 y name2: strings con el nnombre de las variables
    """
    print(f"\n Análisis de homocedasticidad entre '{name1}' y '{name2}'")

    # Prueba de Levene (no asume normalidad)
    levene_stat, levene_p = stats.levene(var1, var2)
    print(f"Levene test: Estadístico = {levene_stat:.4f}, p-valor = {levene_p:.4f}")

def t_test_one(data,mu,variable): #Prueba T para una muestra
    """
    data: arreglo de datos a comparar
    mu: media poblacional o valor de referencia 
    variable: string con el nombre de la variable que se está comparando
    """
    print(f"Prueba T para una sola muestra para {variable}")
    t_stat, p_value = stats.ttest_1samp(data, mu)
    print(f"Estadístico = {t_stat:.4f}, valor_p = {p_value:.4f}")
    
def box_cox(data): #transformación depotencia   

    transformed_data, lambda_opt = stats.boxcox(data)
    return transformed_data, lambda_opt

def tukey(respuesta,factor, alfa,n_factor):
    """

    Parameters
    ----------
    respuesta : Array
        DESCRIPTION. Array con los datos de la variable respuesta
    factor : Array
        DESCRIPTION.Array con los niveles del factor 
    alfa : Float
        DESCRIPTION. Valor alfa de comparación 
    n_factor : String
        DESCRIPTION. Nombre del factor

    Returns
    -------
    None.

    """
    
    tukey = pairwise_tukeyhsd(respuesta, factor, alpha=alfa)
    print(f"Prueba Tukey para el factor {n_factor}")
    print(tukey)
    
def kruskal_W(df,Respuesta,Factor):
    """
    
    Parameters
    ----------
    df : Data_Frame
        DESCRIPTION. estructura con los datos del experimento
    Respuesta : String
        DESCRIPTION. nombre de la variable respuesta, key del dataframe
    Factor : String
        DESCRIPTION. nombre del factor, key del dataframe

    Returns
    -------
    None.

    """
    grupos_B = [df[Respuesta][df[Factor] == nivel] for nivel in df[Factor].unique()]
    stat_B, p_B = stats.kruskal(*grupos_B)
    print(f"Kruskal-Wallis para {Factor}: H = {stat_B:.4f}, p = {p_B:.4f}")
    
    
def kruskal_interaccion(df,Respuesta,Factor1,Factor2):
    """
    

    Parameters
    ----------
    df : Data_Frame
        DESCRIPTION. estructura con los datos del experimento
    Respuesta : String
        DESCRIPTION. nombre de la variable respuesta, key del dataframe
    Factor1 : String
        DESCRIPTION. nombre del factor1, key del dataframe
    Factor2 : String
        DESCRIPTION.nombre del factor12, key del dataframe

    Returns
    -------
    None.

    """
    
    df['interaccion'] = df[Factor1].astype(str) + "_" + df[Factor2].astype(str) # se genera una columana con las combinaciones entre factores

    grupos_interaccion = [df[Respuesta][df['interaccion'] == nivel] for nivel in df['interaccion'].unique()]
    stat_int, p_int = stats.kruskal(*grupos_interaccion)
    print(f"Kruskal-Wallis para la interacción {Factor1}x{Factor2} p = {p_int:.4f}")

In [11]:
df = (pd.read_csv("Proyecto 2/FLIR_groups1and2.csv", sep=";" , header=2))[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4', 
        'Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4',
        'T_Max1', 'T_Max2', 'T_Max3', 'T_Max4',
        'T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4',
        'aveOralM', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Cosmetics','Distance']].copy() #cargamos el dataframe


#creamos la nueva columna con el valor promediado de Max1R13 y eliminamos las demás
df['Max1R13'] = df[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4']].mean(axis=1, skipna=True).astype(float)
df.drop(columns=['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4'],inplace=True) 

#creamos la nueva columna con el valor promediado de Max1L13 y eliminamos las demás
df['Max1L13'] = df[['Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4']].mean(axis=1, skipna=True).astype(float)
df.drop(columns=['Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4'],inplace=True)

#creamos la nueva columna con el valor promediado de T_Max y eliminamos las demás
df['T_Max'] = df[['T_Max1', 'T_Max2', 'T_Max3', 'T_Max4']].mean(axis=1, skipna=True).astype(float)
df.drop(columns=['T_Max1', 'T_Max2', 'T_Max3', 'T_Max4'],inplace=True)

#creamos la nueva columna con el valor promediado de TF_HCC y eliminamos las demás
df['TF_HCC'] = df[['T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4']].mean(axis=1, skipna=True).astype(float)
df.drop(columns=['T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4'],inplace=True)


print(df.isnull().sum()) #Corroboramos si hay datos nulos

#Veamos el comportamiento de los valores nulos de Cosmetics y como podemos tratarlos

        #Primero pasamos los datos a una string pues son categoricos, para poder tratarlos como categoricos 

df["Cosmetics"] = df["Cosmetics"].map({0.0: "No", 1.0: "Sí"})

print(f"\nmoda de cosmeticos: {df['Cosmetics'].mode()}")

b=df['Cosmetics'].fillna(df["Cosmetics"].mode())
print(f"mode de cosmeticos imputando con la moda: {b.mode()}\n")

#procedemos a imputar los valores nulos con la mediana

df["Cosmetics"] = df["Cosmetics"].fillna("No")

df.info()

aveOralM      0
Gender        0
Age           0
Ethnicity     0
T_atm         0
Humidity      0
Cosmetics    29
Distance      2
Max1R13       0
Max1L13       0
T_Max         0
TF_HCC        0
dtype: int64

moda de cosmeticos: 0    No
Name: Cosmetics, dtype: object
mode de cosmeticos imputando con la moda: 0    No
Name: Cosmetics, dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   aveOralM   1020 non-null   float64
 1   Gender     1020 non-null   object 
 2   Age        1020 non-null   object 
 3   Ethnicity  1020 non-null   object 
 4   T_atm      1020 non-null   float64
 5   Humidity   1020 non-null   float64
 6   Cosmetics  1020 non-null   object 
 7   Distance   1018 non-null   float64
 8   Max1R13    1020 non-null   float64
 9   Max1L13    1020 non-null   float64
 10  T_Max      1020 non-null   float64
 11  TF_HCC     1020 non-n